In [1]:
# Given the values of a single peakon on a coarse grid, this DNN predicts the value of the peakon at a particular location.
# Essentially, given the values shown by the @ and G symbols, and a location within the ... region, it will output its
# prediction for the value at that location.
# (In terms of the gen_one_data function below, then (t, x) is the location of the G symbol and (t2, x2) is the random
# location within the ... region.)
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#       .........
#       .       .
# @   @ . @   @ . @   @
#       .       .
#       .       .
#       .       .
# @   @ . G   @ . @   @
#       .       .
#       .........
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
%run base/base_eq.ipynb
%run base/base_plot.ipynb
%run base/base_ml.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [ ]:
class AtPointMixin(Processor):
    """Adapts the Processor transforms to only act on the [:-2] elements of
    X; i.e. omitting the extra (t, x) pair that is given to it.
    """
    def transform(self, X, y):
        X[:-2], y = super(ScaleDataOverallPoint, self).transform(X[:-2], y)
        return X, y
    
    
class ScaleOverallAtPoint(AtPointMixin, ScaleOverall):
    pass
   
    
class NormalisationOverallAtPoint(AtPointMixin, NormalisationOverall):
    pass

In [ ]:
def gen_one_peakon_at_point():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and a particular point, and the 
    label is the value of the single-peakon solution at that particular point.
    """
    point, peakon = gen_one_peakon()
    return sol_at_point(point, peakon)

In [5]:
### DNN hyperparameters
hidden_units = [800] * 6
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
preprocessor = NormalisationOverallAtPoint(momentum=0.999)
activation=tf.nn.crelu

### Training hyperparameters #1
gradient_clip = 1.0  # May be set to None to turn off gradient clipping
drop_rate = 0.0
drop_type = 'dropout'

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/point_single_peakon/'
uuid = None
model_dir = model_dir_str(model_dir_base, hidden_units, logits, 
                          preprocessor=preprocessor, activation=activation, 
                          uuid=uuid)

dnn = create_dnn(hidden_units, logits, drop_rate=drop_rate,
                 drop_type=drop_type, model_dir=model_dir, 
                 log_steps=log_steps, activation=activation, 
                 gradient_clip=gradient_clip, batch_norm=False)
preprocessor.load(model_dir)

max_steps = 0
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 500
    max_steps += 5000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_data, batch_size=batch_size))
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=max_steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

    batch_size = 1000
    max_steps += 5000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_data, batch_size=batch_size))
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=max_steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

    batch_size = 3000
    max_steps += 10000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_data, batch_size=batch_size))
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=max_steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])